In [1]:
from utils import *
from config import *
import pandas as pd
from datetime import datetime

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [3]:
shp_path = '../data/global_wdpa_June2021/Global_wdpa_wInfo_June2021.shp'
gdf = gpd.read_file(shp_path)

In [10]:
wdpaids = gdf['WDPA_PID'].head(15).tolist()
print(wdpaids)

['3', '6', '7', '8', '10', '11', '12', '15', '16', '20', '24', '30', '31', '33', '35']


In [ ]:
geo_ops = GeometryOperations()
img_ops = ImageOperations()
stats_ops = StatsOperations()
viz = Visualization()
feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)
exporter = ExportResults()

In [ ]:
pa = load_local_data('916')
pa_geometry = pa.geometry()
aoi1 = geo_ops.buffer_polygon(pa_geometry, 10000)
aoi1 = geo_ops.mask_water(aoi1)
aoi2 = geo_ops.buffer_polygon(pa_geometry, 1000)

In [ ]:
# Process imagery and add indices
modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi1, 2010))
band_names = modis_ic.first().bandNames()
composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi1)
image = img_ops.add_indices_to_image(composite)

In [ ]:
single_band = image.select('EVI')
gradient = img_ops.get_gradient_magnitude(single_band)

In [ ]:
boundary_pixels = gradient.clip(aoi2)

In [ ]:
buffer_ct = gradient.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=aoi1,
            scale=500,
            maxPixels=1e10)

boundary_ct = boundary_pixels.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=boundary_pixels.geometry(),
            scale=500,
            maxPixels=1e10)

print('Buffer pixel count:', buffer_ct.getInfo())
print('Boundary pixel count:', boundary_ct.getInfo())   

In [14]:
default_vis_params = {
    'min': -0.5,
    'max': 1,
    'palette': ['black', 'gray','white']
}


pa = load_local_data('12')
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(pa.geometry(), 8)
Map.addLayer(pa.geometry(), {'color': 'red'}, 'Buffer')
#Map.addLayer(aoi2, {'color': 'blue'}, 'Boundary')
#Map.addLayer(boundary_pixels, {'color': 'Green'}, 'New')
Map

Map(center=[-25.065945094873772, -58.137524103228756], controls=(WidgetControl(options=['position', 'transpare…